In [1]:
import time
import os

from sklearn.cluster import SpectralClustering
import networkx as nx
import pandas as pd
import numpy as np


In [2]:
def process_file(file_path: str, num_clusters: int | None = None) -> str:
    try:
        adj_matrix = np.loadtxt(file_path, delimiter=",", dtype=int)
        if adj_matrix.shape[0] != adj_matrix.shape[1]:
            raise ValueError("Input matrix must be square.")
    except Exception as e:
        raise ValueError(f"Error reading or validating file {file_path}: {e}")
    
    G = nx.from_numpy_array(adj_matrix)
    
    if num_clusters is None:
        laplacian_matrix = nx.normalized_laplacian_matrix(G).toarray()
        eigenvalues = np.linalg.eigvalsh(laplacian_matrix)
        eigengap = np.diff(eigenvalues)
        num_clusters = np.argmax(eigengap) + 1  # Heurystyka eigengap
    
    clustering = SpectralClustering(
        n_clusters=num_clusters, affinity="precomputed", random_state=42
        )

    labels = clustering.fit_predict(adj_matrix)
    
    results = [
        (node + 1, cluster + 1)
        for node, cluster in enumerate(labels)
        ]

    output_file = (
        "results/" +
        os.path.splitext(os.path.basename(file_path))[0] +
        "_results.csv"
    )

    df_results = pd.DataFrame(results, columns=["Vertex", "Cluster"])
    df_results.to_csv(output_file, index=False, header=False)

    return output_file


In [3]:
os.makedirs("results", exist_ok=True)

files = [
    "data/D1-K=2.csv", "data/D1-UNC.csv",
    "data/D2-K=7.csv", "data/D2-UNC.csv",
    "data/D3-K=12.csv", "data/D3-UNC.csv"
]

description_data = []
for file_path in files:
    start_time = time.time()
    try:
        if "UNC" in file_path:
            num_clusters = None
        else:    
            num_clusters = int(file_path.split("K=")[-1].split(".")[0])

        output_file = process_file(file_path, num_clusters)
        execution_time = time.time() - start_time
        description_data.append((file_path, output_file, execution_time))
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [4]:
with open("results/description.txt", "w") as desc_file:
    desc_file.write("Patrycja Wysocka, Lukasz Jaremek\n")
    desc_file.write("https://github.com/LJaremek/SNARS-Competition-MINI\n")
    for file_path, output_file, exec_time in description_data:
        desc_file.write(f"{file_path}, {output_file}, {exec_time:.2f}s\n")
